In [ ]:
from __future__ import division, unicode_literals, print_function  # for compatibility with Python 2 and 3
import sys
sys.path.append('../src')  # Adjust the path as necessary
# sys.path.append('..')  # Adjust the path as necessary

# from SPTnano import ROISelector, process_directory, generate_file_tree, display_file_tree, overlay_tracks_with_movie, plot_histograms,  read_mat_file, add_microns_and_secs, add_unique_id, plot_particle_trajectory, plot_multiple_particles, filter_stubs
import SPTnano as spt
# from SPTnano import ROISelector, process_directory

master = spt.config.MASTER
saved_data = spt.config.SAVED_DATA

# import pixel size and frame rate
pixelsize_microns = spt.config.PIXELSIZE_MICRONS
time_between_frames = spt.config.TIME_BETWEEN_FRAMES
orderofconditions = spt.config.ORDEROFCONDITIONS
features = spt.config.FEATURES

min_track_length = spt.config.TIME_WINDOW

from IPython.display import Markdown, display

import matplotlib as mpl
import matplotlib.pyplot as plt


import numpy as np
import pandas as pd
from pandas import DataFrame, Series  # for convenience

import pims
import trackpy as tp
import os
import glob
import nd2
import seaborn as sns

# change the following to %matplotlib notebook for interactive plotting
%matplotlib inline

# Optionally, tweak styles.
mpl.rc('figure',  figsize=(10, 5))
mpl.rc('image', cmap='gray')

sns.set_context("notebook", rc={"xtick.labelsize": 10, "ytick.labelsize": 10})


In [ ]:
# Initialize an empty list to store DataFrames
dataframes = []

# Walk through the directory structure
for dirpath, dirnames, filenames in os.walk(master):
    # print(dirpath)
    # print(dirnames)
    # print(filenames)
    for filename in filenames:
        # print(filename)
        if filename.endswith('_tracked.mat'):
            file_path = os.path.join(dirpath, filename)
            # print(file_path)
            df = spt.read_mat_file(file_path)
            # print(file_path)
            # Add a column for the condition and file identifier
            # condition = os.path.basename(os.path.dirname(os.path.dirname(file_path)))
            condition = os.path.basename(os.path.dirname(file_path))


            identifier = os.path.splitext(filename)[0]
            df['condition'] = condition
            df['filename'] = identifier
            #### If you want to get the ms from the condition name, use below ####
            # extract the part of the condition name after the last underscore
            # condition_name = condition.split('_')[-1]
            # # remove the ms from the condition name
            # condition_name = condition_name.replace('ms', '')
            # time_between_frames = int(condition_name) / 1000
            # print(time_between_frames)
            #### If you want to get the ms from the condition name, use above ####
            df = spt.add_microns_and_secs(df, pixelsize_microns, time_between_frames)
            dataframes.append(df)

# # Concatenate all DataFrames into a single DataFrame
df = pd.concat(dataframes, ignore_index=True)
df = spt.add_unique_id(df)


In [ ]:
# # Filter the short tracks out!
df_filtstubs = spt.filter_stubs(df, min_time=0.6) #0.6 for before, 0.2 now

In [ ]:
segment_length_threshold = 1.0 # segment length threshold in um. 
# segment_length_threshold = 0.3

print(segment_length_threshold)
min_track_length_seconds = 0.6  # Specify the minimum track duration in seconds
# time_between_frames = config.TIME_BETWEEN_FRAMES  # From config

# Run the cleaning function
pathfix_df, pathfix_removed_unique_ids, pathfix_report = spt.pathfixer(
    df_filtstubs, 
    segment_length_threshold=segment_length_threshold, 
    min_track_length_seconds=min_track_length_seconds,
    time_between_frames=time_between_frames
)

# saved the cleaned df
pathfix_df.to_csv(saved_data + 'cleaned_df.csv', index=False)

In [ ]:


# Create an instance of ParticleMetrics with your data and time_between_frames value
particle_metrics = spt.ParticleMetrics(pathfix_df, time_between_frames)

# Calculate all features, without calculating time-windowed metrics initially
metrics_df = particle_metrics.calculate_all_features(calculate_time_windowed=False)

# Specify the window size and overlap (in number of frames)
window_size = 60  # e.g. 60 frames per window
overlap = 30      # e.g. 30 frames overlap between windows

entropy_bins = 18          # Use 24 bins for directional entropy
pausing_speed_threshold = 0.1  # Use a speed threshold of 0.2 (units matching your dataset)



particle_metrics.calculate_time_windowed_metrics(
    window_size=60,
    overlap=30,
    allow_partial_window=False,
    min_window_size=60,
    fit_method="r2_threshold",  # use the AIC-based model selection method model_selection or 
    bad_fit_strategy='excise_window',#"excise_window",  # or "remove_track" or "flag"
    r2_threshold=0.000001,
    anomalous_r2_threshold=0.000001,
    use_ci=True,         # use confidence intervals based on uncertainty
    ci_multiplier=1.96,     # for ~95% confidence intervals
        entropy_bins=entropy_bins,
    pausing_speed_threshold=pausing_speed_threshold
)

# Retrieve the DataFrames
time_averaged_df = particle_metrics.get_time_averaged_df()
time_windowed_df = particle_metrics.get_time_windowed_df()
msd_lagtime_df = particle_metrics.msd_lagtime_df




In [ ]:
# saved the dfs to saved data
time_averaged_df.to_csv(saved_data + 'time_averaged_df.csv', index=False)
time_windowed_df.to_csv(saved_data + 'time_windowed_df.csv', index=False)
msd_lagtime_df.to_csv(saved_data + 'msd_lagtime_df.csv', index=False)
# saved the metrics df
metrics_df.to_csv(saved_data + 'metrics_df.csv', index=False)